<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-05-09 15:40:54
-------------------
qualified stocks: 94
with latest results: 87
still star stocks: 47
-------------------
Initial Investment:  1.14 C
CY Investment:  1.41 C
Reserve:  3.78 L
Current:  1.22 C
-------------------
Today PnL: -47.97 K (-0.39%)
Current PnL: -23.97 L (-17.0%)
CY Booked + Current PnL: -21.13 L (-14.99%)
-------------------
Total profit:  2.49 L
Total loss:  -26.46 L
-------------------
Total Booked + Current PnL: 5.85 L (5.13%)
Total Booked PnL: 29.81 L (26.14%)
Curr Year Booked PnL: 2.83 L (2.33%)
Prev Year Booked PnL: 26.98 L (23.66%)
Est FTT:  2.07 C
Est FTT PnL: 85.08 L (69.95%)
-------------------
Est LTT:  2.68 C
Est LTT PnL: 1.47 C (120.69%)
Deployed:  1.14 C
Current:  1.22 C
CAGR/XIRR %: 4.65%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

Symbol    AvgCost        FTT  Dev%_PE Conviction    RSP   Current  \
52         MRF  114777.60  150504.14    19.83       X-MC  96.81  137675.0   
57  PIDILITIND    2504.06    3576.00   -14.36       X-LC  40.43   89400.0   
70     SBILIFE    1496.49    1924.99    -3.88       H-LC  87.23  142783.0   
71        SBIN     760.30     863.00   -17.60       M-LC  61.70  206501.0   
55        NMDC      65.62      82.00    27.19       M-MC  60.64   98364.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
52      22897.0  12831.0       -0.78         19.95   9.32  31.13    203.0   
57      14278.0  17880.0        0.76         19.01  20.00  42.81     66.0   
70      17078.0  18919.0       -1.83         13.59  13.25  28.63    116.0   
71       5022.0  22199.0        1.33          2.49  10.75  13.51    106.0   
55      -2035.0  27099.0        0.78         -2.03  27.55  24.96     70.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy   Category  
52     1.78        1.17    34.11       AR      ATH      TYRES  
57     0.80        0.76    12.12      X40      BTT  CHEMICALS  
70     0.90        1.21    27.20       AR      ATH  INSURANCE  
71     0.23        1.75    13.13     XY25      NTT      BANKS  
55    -0.08        0.83    10.86     XY25      NTT     MINING

In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost       FTT  Dev%_PE Conviction     RSP   Current  \
87   UNIONBANK   123.87    163.00   -12.71       M-LC   65.43  139737.0   
19        CERA  9059.90  10420.45    89.27       X-SC   77.66   58320.0   
90      VIPIND   488.80    718.00  -776.08       H-SC  100.00   64438.0   
81  TATAMOTORS   776.02   1065.00   -53.26       M-LC   95.74  187752.0   
2          ABB  5319.37   8953.12   -40.07       H-LC   81.91  136012.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%    FTT%   OTT%  CumlRnk  \
87      -1103.0  45596.0        6.33         -0.78   32.63  31.59    102.0   
19     -32279.0  45886.0        5.63        -35.63   78.68  15.02     50.0   
90     -30878.0  75573.0        4.56        -32.40  117.28  46.89     39.0   
81     -17893.0  94477.0        3.76         -8.70   50.32  37.24      1.0   
2        3028.0  87809.0        2.89          2.28   64.56  68.31      3.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
87    -0.02        1.19    21.38     XY24      NTT       BANKS  
19    -0.70        0.49    14.75     X40N      ATH    CERAMICS  
90    -0.41        0.55    27.10     X40N      NTT        MISC  
81    -0.19        1.59    22.21     XY24      NTT        AUTO  
2      0.03        1.15    10.91       AR      ATH  ELECTRICAL

In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction    RSP   Current  \
78    SURYODAY   189.78   240.00    27.19       H-SC  45.74   90818.0   
84     TRIDENT    37.35    48.00    37.49       M-SC  62.77   66048.0   
24         DLF   683.35  1031.70   -31.48       H-LC  52.13  113706.0   
65       ROUTE  1558.40  2249.26   -13.23       M-SC   9.57   60218.0   
89  VALIANTORG   512.64   838.00  -276.54       H-SC   2.13   74958.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%   OTT%  CumlRnk  \
78     -59108.0   98783.0       -6.45        -39.42  108.77  26.46    115.0   
84     -26206.0   52515.0       -3.88        -28.41   79.51  28.51    202.0   
24      -9297.0   71999.0       -3.56         -7.56   63.32  50.98      4.0   
65     -42636.0   88231.0       -3.26        -41.45  146.52  44.33     19.0   
89     -58328.0  142922.0       -3.11        -43.76  190.67  63.47    106.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy   Category  
78    -0.60        0.77    16.34       XR      NTT      BANKS  
84    -0.50        0.56    11.32       XR      NTT   TEXTILES  
24    -0.13        0.96     3.89       AR      ATH     REALTY  
65    -0.48        0.51     0.00       SR      ATH         IT  
89    -0.41        0.64    25.65       XR      NTT  CHEMICALS

In [12]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction    RSP   Current  \
81  TATAMOTORS   776.02  1065.00   -53.26       M-LC  95.74  187752.0   
79       TANLA   917.30  1963.11   -58.95       H-SC  54.26  122709.0   
2          ABB  5319.37  8953.12   -40.07       H-LC  81.91  136012.0   
24         DLF   683.35  1031.70   -31.48       H-LC  52.13  113706.0   
10         AWL   342.88   485.00   -70.43       H-MC   7.45  160401.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%    OTT%  CumlRnk  \
81     -17893.0   94477.0        3.76         -8.70   50.32   37.24      1.0   
79    -116706.0  389662.0       -1.63        -48.75  317.55  114.01      2.0   
2        3028.0   87809.0        2.89          2.28   64.56   68.31      3.0   
24      -9297.0   71999.0       -3.56         -7.56   63.32   50.98      4.0   
10     -57328.0  147569.0       -0.67        -26.33   92.00   41.45      6.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy    Category  
81    -0.19        1.59    22.21     XY24      NTT        AUTO  
79    -0.30        1.04    12.54       AR      ATH          IT  
2      0.03        1.15    10.91       AR      ATH  ELECTRICAL  
24    -0.13        0.96     3.89       AR      ATH      REALTY  
10    -0.39        1.36     7.15     XY24      NTT        FMCG

In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

Empty DataFrame
Columns: [Symbol, AvgCost, FTT, Dev%_PE, Conviction, RSP, Current, Current P/L, FTT Amt, Today P/L%, Current P/L%, FTT%, OTT%, CumlRnk, RRR Ind, CurrAlloc%, Gained%, Criteria, Strategy, Category]
Index: []

In [14]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction    RSP   Current  \
58     POLYCAB  5005.92  7541.35    13.66       H-MC  89.36  161462.0   
4   ADANIPORTS  1103.69  1583.00     1.42       M-LC  90.43  178963.0   
56      PFIZER  4236.33  6318.30   -15.37       H-SC  67.02  131747.0   
71        SBIN   760.30   863.00   -17.60       M-LC  61.70  206501.0   
92   WHIRLPOOL  1151.42  2270.00   -38.32       M-SC  86.17  155308.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
58      21296.0   49698.0        0.03         15.19  30.78  50.65     74.0   
4       27757.0   37904.0       -2.01         18.36  21.18  43.43     45.0   
56        421.0   64121.0       -0.31          0.32  48.67  49.15     89.0   
71       5022.0   22199.0        1.33          2.49  10.75  13.51    106.0   
92       9078.0  132975.0       -0.98          6.21  85.62  97.15      8.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy  Category  
58     0.43        1.37    23.23     X40N      ATH    CABLES  
4      0.73        1.52    24.05     XY24      NTT      MISC  
56     0.01        1.12     8.59      X40      ATH    PHARMA  
71     0.23        1.75    13.13     XY25      NTT     BANKS  
92     0.07        1.32    32.68      X40      NTT  DURABLES

In [15]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

Symbol  AvgCost      FTT  Dev%_PE Conviction    RSP   Current  \
93      WIPRO   243.46   420.00   -12.43       M-LC  47.87  150046.0   
12  BANKINDIA   113.49   190.00   -28.40       M-MC  53.19  125430.0   
91     VOLTAS  1272.00  1929.20   -20.25       H-MC  14.89  135916.0   
1    AARTIIND   487.04   919.00    19.22       M-SC  98.94   75417.0   
18       CAMS  3643.00  5250.99   -11.88       H-SC  17.02   97429.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%    FTT%   OTT%  CumlRnk  \
93       -899.0  110359.0        0.18         -0.60   73.55  72.51     77.0   
12      -3722.0   90786.0        2.19         -2.88   72.38  67.42     99.0   
91      -4004.0   76290.0        1.02         -2.86   56.13  51.67      9.0   
1       -5919.0   78057.0        0.68         -7.28  103.50  88.69    114.0   
18      -4575.0   49601.0       -2.51         -4.49   50.91  44.14     26.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy   Category  
93    -0.01        1.27    12.97       XR      NTT         IT  
12    -0.04        1.06    21.99       XR      NTT      BANKS  
91    -0.05        1.15     2.66       AR      ATH         AC  
1     -0.08        0.64    27.18       XR      NTT  CHEMICALS  
18    -0.09        0.83    13.99       XR      ATH       MISC

In [16]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

Symbol    AvgCost        FTT  Dev%_PE Conviction    RSP   Current  \
86  UJJIVANSFB      52.77      60.00   -43.55       M-SC  58.51  109971.0   
52         MRF  114777.60  150504.14    19.83       X-MC  96.81  137675.0   
70     SBILIFE    1496.49    1924.99    -3.88       H-LC  87.23  142783.0   
61     RBLBANK     230.61     388.00    29.03       M-SC  92.55  199969.0   
69    SBIETFPB     241.24     400.00      NaN        NaN  74.47  205798.0   

    Current P/L   FTT Amt  Today P/L%  Current P/L%   FTT%   OTT%  CumlRnk  \
86     -32508.0   52027.0       -1.71        -22.82  47.31  13.70    139.0   
52      22897.0   12831.0       -0.78         19.95   9.32  31.13    203.0   
70      17078.0   18919.0       -1.83         13.59  13.25  28.63    116.0   
61     -34331.0  194230.0        0.23        -14.65  97.13  68.25    207.0   
69      23179.0   96993.0       -1.07         12.69  47.13  65.81      NaN   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy   Category  
86    -0.62        0.93    29.80     X40N      NTT      BANKS  
52     1.78        1.17    34.11       AR      ATH      TYRES  
70     0.90        1.21    27.20       AR      ATH  INSURANCE  
61    -0.18        1.70    31.69     XY24      NTT      BANKS  
69     0.24        1.75    14.92     XY24      BTT      BANKS

In [17]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

Symbol    AvgCost        FTT  Dev%_PE Conviction     RSP   Current  \
90      VIPIND     488.80     718.00  -776.08       H-SC  100.00   64438.0   
1     AARTIIND     487.04     919.00    19.22       M-SC   98.94   75417.0   
76    SONACOMS     578.05    1006.00   -32.65       M-MC   97.87   87588.0   
52         MRF  114777.60  150504.14    19.83       X-MC   96.81  137675.0   
81  TATAMOTORS     776.02    1065.00   -53.26       M-LC   95.74  187752.0   

    Current P/L  FTT Amt  Today P/L%  Current P/L%    FTT%   OTT%  CumlRnk  \
90     -30878.0  75573.0        4.56        -32.40  117.28  46.89     39.0   
1       -5919.0  78057.0        0.68         -7.28  103.50  88.69    114.0   
76     -13571.0  88464.0       -1.50        -13.42  101.00  74.03      7.0   
52      22897.0  12831.0       -0.78         19.95    9.32  31.13    203.0   
81     -17893.0  94477.0        3.76         -8.70   50.32  37.24      1.0   

    RRR Ind  CurrAlloc%  Gained% Criteria Strategy   Category  
90    -0.41        0.55    27.10     X40N      NTT       MISC  
1     -0.08        0.64    27.18       XR      NTT  CHEMICALS  
76    -0.15        0.74    23.09       AR      BTT       AUTO  
52     1.78        1.17    34.11       AR      ATH      TYRES  
81    -0.19        1.59    22.21     XY24      NTT       AUTO

In [18]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

Top_N  Sum_Alloc%
0     10       19.54
1     25       41.47
2     50       69.91

In [19]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    35.72
X40     14.69
X40N    13.74
AR      12.17
XY25    11.23
XR      10.17
X200     1.18
SR       1.11
Name: CurrAlloc%, dtype: float64

In [20]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    21.27
X-LC    18.57
M-SC    16.35
M-LC     8.45
M-MC     7.44
X-MC     6.84
H-MC     6.83
H-LC     5.94
X-SC     2.59
L-SC     2.21
Name: CurrAlloc%, dtype: float64

In [21]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

CurrAlloc%  Current P/L%    FTT%
Category                                       
FMCG                11.22         -4.50   37.77
BANKS               11.09        -15.34   71.92
IT                   9.12        -28.69  100.49
FINANCE              7.31        -28.62   86.66
MISC                 6.31        -38.70   88.88
INSURANCE            6.05         -8.30   40.95
PAINTS               5.74        -21.64   57.64
ELECTRICAL           4.15        -13.60   71.48
REFINERIES           3.69          6.44   10.80
HEALTHCARE           3.66         -7.89   42.73
AUTO                 3.23        -19.16   72.50
CHEMICALS            3.22        -24.60  110.45
FOOTWEAR             2.44        -43.30   90.65
DURABLES             2.00         -4.87   86.32
POWER                1.99        -20.57   68.79
AC                   1.97        -15.08   76.47
TRAVEL               1.93        -39.79  100.28
TEXTILES             1.65        -18.45   80.66
STEEL                1.59        -19.84  109.16
CERAMICS             1.47        -64.01  130.73
CABLES               1.37         13.19   30.78
JEWELLERY            1.23       -100.62  164.05
TYRES                1.17         16.63    9.32
CEMENT               1.12        -42.36  115.61
PHARMA               1.12          0.32   48.67
ENGINEERING          1.02        -59.65  125.25
REALTY               0.96         -8.18   63.32
ENTERTAINMENT        0.88        -57.57  136.81
MINING               0.83         -2.07   27.55
HOTELS               0.48       -100.41  171.09

In [22]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Criteria                  
XY24      3340644.0     28
XR        1346996.0     15
X40N      1280675.0     17
AR        1169806.0     13
X40        863645.0     12
SR         214468.0      2
XY25       212060.0      6
X200        79223.0      1

In [23]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction                  
H-SC        2468569.0     21
M-SC        2078799.0     20
X-LC         796260.0     12
M-MC         713013.0      7
H-MC         422403.0      6
M-LC         384576.0      6
X-SC         372484.0      4
X-MC         348534.0      6
L-SC         336591.0      4
H-LC         258642.0      5

In [24]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      977672.0      6
M-SC       XY24      831705.0      6
H-SC       X40N      597597.0      7
M-MC       XY24      506664.0      4
M-SC       XR        456673.0      5
H-SC       AR        452990.0      2
X-LC       X40       402617.0      5
M-SC       X40N      398491.0      5
H-SC       XR        333291.0      4
M-LC       XY24      252018.0      4
M-SC       X40       207691.0      2
X-LC       AR        195569.0      3
H-MC       XY24      193020.0      2
H-LC       AR        178727.0      3
L-SC       XR        169897.0      2
X-MC       XY24      154152.0      2
X-SC       XR        130026.0      1
           SR        126237.0      1
X-LC       XY25      118851.0      3
X-MC       X40N      118668.0      2
X-SC       X40N      116221.0      2
M-LC       XR        110359.0      1
H-SC       X40       107019.0      2
L-SC       XY24       97767.0      1
M-SC       AR         96008.0      1
M-MC       XR         90786.0      1
           AR         88464.0      1
M-SC       SR         88231.0      1
X-LC       X200       79223.0      1
H-MC       AR         76290.0      1
L-SC       AR         68927.0      1
X-MC       X40        62883.0      1
H-MC       XR         55964.0      1
           X40N       49698.0      1
           X40        47431.0      1
H-LC       XY25       43911.0      1
           X40        36004.0      1
M-MC       XY25       27099.0      1
M-LC       XY25       22199.0      1
X-MC       AR         12831.0      1

In [25]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 28.0 seconds
